In [ ]:
import torch
import torchvision
from torchvision.models.detection import 
#I havent sanitized the imports, it is a but much now. But whatever...
from torchvision.transforms.v2 import functional as F
from torchvision.transforms import v2
from torchvision import ops
import torchvision.transforms.v2 as transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import datapoints
from torchvision.ops import generalized_box_iou_loss
from torch.utils import tensorboard
from torchvision.datasets import VisionDataset


#in models/ there are quite a few models, saved in the intermediate training steps. 
#following: ssdlite_cows_model_exxx.pth hwere xxx is n epoch
# the latest is: models/ssdlite_cows_model_500e.pth (note the switched e)


# loss on val-set hovered around the 0.01 range from epoch 200 onward
# (which I think was calculated with cross-entropy and class accuracy afaik (but only 1 class: cows, with label: 0)
# Create your SSD Lite model without any pretrained weights:

model = ssdlite320_mobilenet_v3_large()
# but then like overwrite it with the local state_dict etc.
model.load_state_dict(torch.load("models/ssdlite_cows_model_500e.pth"))
model.eval()

In [ ]:
class CustomDataset(VisionDataset):
    def __init__(self, images, labels, boxes, transform=None):
        self.images = images
        self.labels = labels
        self.boxes = boxes
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        labels = torch.tensor(self.labels[idx], dtype=torch.long)
        boxes = torch.tensor(self.boxes[idx], dtype = torch.float32)
        target = {'boxes': boxes,
                 'labels': labels}
        
        if self.transform:
            image = self.transform(image)

        return image, target

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

# Create the dataset
test_dataset = torch.load("data/test_set.pkl")
# Create a DataLoader for the VOC dataset
batch_size = 8 # server is not that amazing to deal with larger batch sizes (8 seems to work with OOM)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn = collate_fn)

In [ ]:
# predict on test set

#check mAP, mIoU etc.

#repeat for various quantization methodologies

In [ ]:
# i guess would also be fun to check if total amount of cows would be correctly counted per img.
# we could also try to predict on the complete image: current test_loader uses the pickled Dataset with 320x320 sliced images